## ANOVAs within each study

In [1]:
import os, sys
sys.path.append(os.path.dirname(os.path.realpath("./"))) # Add parent to path

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
from scipy.stats import linregress
from plots import bars, plot_linreg
import pingouin as pg

import statsmodels.api as sm
from statsmodels.formula.api import ols

import warnings
warnings.filterwarnings('ignore')

In [2]:
plt.style.use("jg.mplstyle")

In [3]:
df_trials = pd.read_pickle('./frames/df_trials.pickle')
df_subjs = pd.read_pickle('./frames/df_subjs.pickle')

In [4]:
def anovas(map_kind="random", lib="pingouin", dv='oracle_requested', anova_type="rm", df=None, factors=None, btwn_factors=None):
    if df is None:
        df = df_trials[df_trials.map_kind == map_kind].copy()
    
    if lib == "pingouin":
        anova_fn = {
            "rm": pg.rm_anova,
            "mixed": pg.mixed_anova
        }[anova_type]
        kwargs = {}
        if anova_type == "rm":
            kwargs['detailed'] = True
            kwargs['effsize'] = 'ng2'            
        elif anova_type == "mixed":
            kwargs['between'] = btwn_factors
            kwargs['effsize'] = 'ng2'
        aov = anova_fn(df, dv=dv, subject="user_id", within=factors, **kwargs)
        posthocs = pg.pairwise_tests(df, dv=dv, subject="user_id", within=factors, between=btwn_factors)
        
        if anova_type == "rm":
            for factor in factors:
                print(df.groupby(factor)[dv].mean())
            print(df.groupby(factors)[dv].mean())
        return aov, posthocs
    else:
        #perform two-way ANOVA
        model = ols('%(dv)s ~ C(%(f1)s) + C(%(f2)s) + C(%(f1)s):C(%(f2)s)' % ({'dv': dv, 'f1': factors[0], 'f2': factors[1]}), data=df).fit()
        return (sm.stats.anova_lm(model, typ=2), None)
    

## Study A ANOVAS

In [5]:
model, posthocs = anovas(factors=['cond_best_strat', 'cond_gap_size'])

cond_best_strat
NO    0.061607
OF    0.558929
Name: oracle_requested, dtype: float64
cond_gap_size
L    0.348214
S    0.272321
Name: oracle_requested, dtype: float64
cond_best_strat  cond_gap_size
NO               L                0.041071
                 S                0.082143
OF               L                0.655357
                 S                0.462500
Name: oracle_requested, dtype: float64


In [6]:
model

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,cond_best_strat,6.925201,1,27,6.925201,67.920661,7.554101e-09,7.554101e-09,0.559777,1.0
1,cond_gap_size,0.161272,1,27,0.161272,22.404387,6.236825e-05,6.236825e-05,0.028760,1.0
2,cond_best_strat * cond_gap_size,0.383058,1,27,0.383058,35.965769,2.131618e-06,2.131618e-06,0.065713,1.0


In [7]:
posthocs

,Contrast,cond_best_strat,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,cond_best_strat,-,NO,OF,True,True,-8.241399,27.0,two-sided,7.554101e-09,1.686e+06,-2.287284
1,cond_gap_size,-,L,S,True,True,4.733327,27.0,two-sided,6.236825e-05,403.043,0.501409
2,cond_best_strat * cond_gap_size,NO,L,S,True,True,-2.820834,27.0,two-sided,8.871375e-03,5.099,-0.520954
3,cond_best_strat * cond_gap_size,OF,L,S,True,True,5.915404,27.0,two-sided,2.644750e-06,7230.63,0.617593


In [8]:
df_retry = df_trials[(df_trials.map_kind == "random") & (df_trials.retry_response > -1)  & (df_trials.cond_oracle_acc == "A")]
model, posthocs = anovas(dv="retry_response", df=df_retry, factors=['oracle_requested', 'correct_strat'])

oracle_requested
0.0    0.745455
1.0    0.946237
Name: retry_response, dtype: float64
correct_strat
0.0    0.568966
1.0    0.890000
Name: retry_response, dtype: float64
oracle_requested  correct_strat
0.0               0.0              0.500000
                  1.0              0.846154
1.0               0.0              0.900000
                  1.0              0.951807
Name: retry_response, dtype: float64


In [9]:
model

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,oracle_requested,0.907500,1,2,0.907500,21.671642,0.043177,0.043177,0.684475,1.0
1,correct_strat,0.140833,1,2,0.140833,2.245847,0.272709,0.272709,0.251863,1.0
2,oracle_requested * correct_strat,0.140833,1,2,0.140833,2.245847,0.272709,0.272709,0.251863,1.0


In [10]:
posthocs

,Contrast,oracle_requested,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,oracle_requested,-,0.0,1.0,True,True,-4.655281,2.0,two-sided,0.043177,2.791,-3.040817
1,correct_strat,-,0.0,1.0,True,True,-1.498615,2.0,two-sided,0.272709,0.871,-1.071966
2,oracle_requested * correct_strat,0.0,0.0,1.0,True,True,-1.498615,2.0,two-sided,0.272709,0.871,-1.071966
3,oracle_requested * correct_strat,1.0,0.0,1.0,True,True,NaN,2.0,two-sided,NaN,nan,NaN


In [17]:
df_responded = df_trials[(df_trials.retry_response > -1) & (df_trials.map_kind == "random")]
posthocs = pg.pairwise_tests(df_responded, dv='retry_response', subject="user_id", within='cond_oracle_acc')
posthocs

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,cond_oracle_acc,A,I,True,True,1.707865,27.0,two-sided,0.099136,0.723,0.176661


## Study B ANOVAS

In [11]:
# Populate ius-type in df trials
ius_median = np.median(df_subjs.ius_score)
for key, row in df_subjs.iterrows():
    ius_type = 'high' if row.ius_score > ius_median else 'low'
    df_subjs.loc[key, 'ius_type'] = ius_type

for key, row in df_trials.iterrows():
    df_trials.loc[key, 'ius_type'] = df_subjs[df_subjs.user_id == row.user_id].iloc[0].ius_type

In [12]:
df_spatial = df_trials[df_trials.map_kind == "spatial"]
model, posthocs = anovas(df=df_spatial, dv="oracle_requested", anova_type="mixed", factors='cond_best_strat', btwn_factors='ius_type')

In [13]:
model

,Source,SS,DF1,DF2,MS,F,p-unc,ng2,eps
0,ius_type,0.258927,1,27,0.258927,6.130995,1.984515e-02,0.115100,NaN
1,cond_best_strat,4.982759,1,27,4.982759,158.206293,8.394411e-13,0.714537,1.0
2,Interaction,0.288743,1,27,0.288743,9.167793,5.366101e-03,0.126675,NaN


In [14]:
posthocs

,Contrast,cond_best_strat,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,cond_best_strat,-,NO,OF,True,True,-11.067000,28.000000,two-sided,9.806397e-12,9.018e+08,-2.716375
1,ius_type,-,high,low,False,True,2.647572,26.972729,two-sided,1.337427e-02,4.106,0.907400
2,cond_best_strat * ius_type,NO,high,low,False,True,-0.273880,26.536475,two-sided,7.862972e-01,0.363,-0.092409
3,cond_best_strat * ius_type,OF,high,low,False,True,3.058055,26.926378,two-sided,4.990169e-03,8.637,1.044700
